# CRAWL PRODUCTS'S INFORMATION

In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from time import sleep
import random
from webdriver_manager.chrome import ChromeDriverManager

In [ ]:
def baxia_dialog_close(driver):
    try:
        # Kiểm tra nếu phần tử 'content-block sfo' có hiện diện trên trang
        dialog_close = driver.find_element(By.CSS_SELECTOR, '.baxia-dialog-close')
        
        if dialog_close.is_displayed():  # Kiểm tra xem phần tử có hiển thị hay không
            print("Phần tử 'baxia-dialog-close' hiện ra. Đang nhấn nút close...")
            
            # Tìm nút đóng (close) trong phần tử hiện ra
            close_button = driver.find_element(By.CSS_SELECTOR, '.baxia-dialog-close')
            
            # Nhấn nút
            close_button.click()
            print("Đã nhấn nút close")
    
    except NoSuchElementException:
        pass

In [ ]:
chrome_options = Options()
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
df = pd.DataFrame(columns=['titles', 'links', 'prices', 'discounts', 'countReviews', 'countSales'])

In [ ]:
def scrape_page(page_number):
    driver.get(f"https://www.lazada.vn/catalog/?page={page_number}&q=đồ%20bộ%20nữ")
    if page_number == 0:
        sleep(15)
    else:
        sleep(random.randint(2, 5))
    sleep(random.randint(2, 5))  # Đợi một chút để trang tải xong
    try:
        # Lấy liên kết và tiêu đề
        elems = driver.find_elements(By.CSS_SELECTOR, ".RfADt [href]")
        titles = [elem.text for elem in elems]
        links = [elem.get_attribute('href') for elem in elems]

        # Lấy giá
        elems_price = driver.find_elements(By.CSS_SELECTOR, ".aBrP0")
        prices = [elem.text for elem in elems_price]

        # Lấy phần trăm giảm giá (discount)
        elems_discount = driver.find_elements(By.CSS_SELECTOR, ".WNoq3")
        discounts = [elem.text for elem in elems_discount]

        # Lấy số lượng đánh giá
        elems_countReviews = driver.find_elements(By.CSS_SELECTOR, ".qzqFw")
        countReviews = [elem.text for elem in elems_countReviews]

        # Lấy số lượng bán
        elems_countSales = driver.find_elements(By.CSS_SELECTOR, "._1cEkb")
        countSales = [elem.text for elem in elems_countSales]
        
        df_new = pd.DataFrame(list(zip(titles , links, prices, discounts, countReviews, countSales)), 
                              columns = ['titles', 'links','prices', 'discounts', 'countReviews', 'countSales'])

        # Bổ sung dữ liệu thiếu
        default_length = len(titles)
        
        if len(prices) < default_length:
            prices.extend(["0"] * (default_length - len(prices)))
        if len(discounts) < default_length:
            discounts.extend(["0%"] * (default_length - len(discounts)))
        if len(countReviews) < default_length:
            countReviews.extend(["0"] * (default_length - len(countReviews)))
        if len(countSales) < default_length:
            countSales.extend(["0"] * (default_length - len(countSales)))

        # Loại bỏ dữ liệu trùng lặp
        df_new = df_new.drop_duplicates(subset=['titles', 'links'], keep='first')

        # Nối df_new với df ban đầu
        global df
        df = pd.concat([df, df_new], ignore_index=True)

        return df

    except NoSuchElementException as e:
        print(f"Element not found on page {page_number}: {e}")
        return None

In [ ]:
def main():
    page_number = 0
    max_pages = 50  # Số lượng trang tối đa muốn cào
    all_data = []

    try:
        while page_number <= max_pages:
            df = scrape_page(page_number)
            baxia_dialog_close(driver)
            if df is not None:
                all_data.append(df)
            page_number += 1
            sleep(random.randint(2, 5))  # Đợi trước khi chuyển sang trang tiếp theo

    finally:
        driver.quit()  # Đảm bảo đóng trình duyệt dù có xảy ra lỗi

    if all_data:
        # Kết hợp tất cả DataFrame thành một DataFrame duy nhất
        final_df = pd.concat(all_data, ignore_index=True)

        # Loại bỏ dữ liệu trùng lặp trong DataFrame cuối cùng
        final_df = final_df.drop_duplicates(subset=['titles', 'links'], keep='first')
    else:
        print("No data was collected.")
        return None

    # Hiển thị DataFrame
    print(final_df)
    
    return final_df

if __name__ == "__main__":
    final_df = main()

In [ ]:
final_df.to_csv(r'C:\Users\ASUS\Desktop\T\ĐAN_KLTN\getdata\thoi_trang_nu.csv', mode='a', index=False, encoding='utf-8-sig')

In [ ]:
input_path = r'C:\Users\ASUS\Desktop\T\ĐAN_KLTN\getdata\thoi_trang_nu.csv'
output_path = r'C:\Users\ASUS\Desktop\T\ĐAN_KLTN\getdata\combined_data.csv'

df = pd.read_csv(input_path)

df_cleaned = df.drop_duplicates()
df_cleaned = df_cleaned.reset_index(drop=True)

df_cleaned['countReviews'] = df_cleaned['countReviews'].astype(str).str.extract(r'(\d+)').astype(float).fillna(0).astype(int)
df_cleaned['countSales'] = df_cleaned['countSales'].astype(str).str.extract(r'(\d+)').astype(float).fillna(0).astype(int)
df_cleaned['prices'] = df_cleaned['prices'].astype(str).str.replace(r'[^\d]', '', regex=True).replace('', '0').astype(float).astype(int)

df_cleaned

In [ ]:
df_cleaned.to_csv(output_path, index=False, encoding='utf-8-sig')
print(f"Đã xử lý và lưu vào: {output_path}")

# EXTRACT IMAGES AND REVIEWS FROM CRAWLED URLS ABOVE

In [1]:
import pandas as pd
import os
import requests
import time
import random
from time import sleep
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
df = pd.read_csv(r"C:\Users\ASUS\Desktop\T\ĐAN_KLTN\getdata\combined_data.csv")

In [3]:
chrome_options = Options()
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

In [4]:
df4 = pd.DataFrame(columns=['name_comment', 'content_comment', 'skuInfo_comment', 'like_count', 'star_count', 'Link', 'stt'])

In [5]:
folder_path = r"C:\Users\ASUS\Desktop\T\ĐAN_KLTN\getImages"

In [6]:
not_found_imgs = []

In [7]:
def content_block (driver):
    try:
        middleware_element = driver.find_element(By.CSS_SELECTOR, 'body > div:nth-child(73) > div > div.next-overlay-inner.animated.expandInDown.overlay-content.pdp-overlay.pdp-overlay_theme_sfo.next-position-tr > div')
        
        if middleware_element.is_displayed():
            print("Phần tử 'content_block' hiện ra. Đang nhấn nút close...")
            
            # Tìm nút đóng (close) trong phần tử hiện ra
            close_button = driver.find_element(By.CSS_SELECTOR, "body > div:nth-child(73) > div > div.next-overlay-inner.animated.expandInDown.overlay-content.pdp-overlay.pdp-overlay_theme_sfo.next-position-tr > div > span > i")
            close_button.click()      
            print("Đã click đóng")
            
    except NoSuchElementException:
        pass

In [8]:
def close_sfo_popup(driver):
    try:
        sfo_element = driver.find_element(By.CSS_SELECTOR, '.content-block.sfo')
        if sfo_element.is_displayed():
            print("Phần tử 'content-block sfo' hiện ra. Đang nhấn nút close...")
            close_button = driver.find_element(By.CSS_SELECTOR, 'i.next-icon.next-icon-close.next-icon-small')
            close_button.click()
            print("Đã nhấn nút close sfo.")
    except NoSuchElementException:
        pass

In [9]:
def baxia_dialog_close(driver):
    try:
        dialog_close = driver.find_element(By.CSS_SELECTOR, '.baxia-dialog-close')
        if dialog_close.is_displayed():
            print("Phần tử 'baxia-dialog-close' hiện ra. Đang nhấn nút close...")
            close_button = driver.find_element(By.CSS_SELECTOR, '.baxia-dialog-close')
            close_button.click()
            print("Đã nhấn nút close")
    except NoSuchElementException:
        pass

In [10]:
def middleware_frame_widget(driver):
    try:
        middleware_element = driver.find_element(By.CSS_SELECTOR, '.J_MIDDLEWARE_FRAME_WIDGET')
        if middleware_element.is_displayed():
            print("Phần tử 'middleware_element' hiện ra. Đang nhấn nút close...")
            close_button = driver.find_element(By.CSS_SELECTOR, "body > div.J_MIDDLEWARE_FRAME_WIDGET > img")
            close_button.click()      
            print("Đã click đóng")
    except NoSuchElementException:
        pass

In [11]:
def download_image(image_url, save_path):
    try:
        response = requests.get(image_url, stream=True)
        if response.status_code == 200:
            with open(save_path, 'wb') as file:
                for chunk in response.iter_content(1024):
                    file.write(chunk)
            print(f"Đã lưu hình ảnh vào: {save_path}")
        else:
            print(f"Không thể tải hình ảnh từ: {image_url}")
    except Exception as e:
        print(f"Lỗi khi tải hình ảnh: {e}")

In [12]:
def scrape_product_image(driver, url, no, folder_path):
    try:
        # Nhấn vào ảnh nhỏ để mở popup
        image_element_small = driver.find_element(By.CSS_SELECTOR, "#module_item_gallery_1 > div > div.next-slick.next-slick-outer.next-slick-horizontal.item-gallery-slider > div > div.next-slick-list > div > div:nth-child(2) > div > img")
        image_element_small.click()
        sleep(4)  # Đợi popup hiển thị
        
        image_element = driver.find_element(By.CSS_SELECTOR, "#module_item_gallery_1 > div > div.gallery-preview-panel > div > img")
        image_url = image_element.get_attribute('src')
        
        image_name = f"{no}.jpg"
        save_path = os.path.join(folder_path, image_name)
        download_image(image_url, save_path)
        return image_name  # Trả về tên hình ảnh
    except NoSuchElementException:
        print(f"Không tìm thấy ảnh cho sản phẩm: {url}")
        not_found_imgs.append(url)
    except Exception as e:
        print(f"Lỗi khi lấy ảnh cho sản phẩm {url}: {e}")
    return None

In [13]:
def count_rate_star(containerStar_element):
    goldStar_url = 'https://img.lazcdn.com/g/tps/tfs/TB19ZvEgfDH8KJjy1XcXXcpdXXa-64-64.png'
    countStar = 0
    img_elements = containerStar_element.find_elements(By.TAG_NAME, 'img')
    for img in img_elements:
        if img.get_attribute('src') == goldStar_url:
            countStar += 1
    return countStar

In [14]:
def scroll_to_comment(driver, element_css_selector, product_link, stt):
    scroll_pause_time = 0.1  
    scroll_step = 10        
    max_wait_time = 30      
    start_time = time.time() 

    while True:
        try:
            if time.time() - start_time > max_wait_time:
                print(f"Vượt quá thời gian {max_wait_time} giây mà không tìm thấy thêm bình luận, dừng lại.")
                break
        
            element = driver.find_element(By.CSS_SELECTOR, element_css_selector)
            driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'center'});", element)
            
            elems_name = driver.find_elements(By.CSS_SELECTOR, "div.middle")
            name_comment = [elem.text for elem in elems_name]

            elems_content = driver.find_elements(By.CSS_SELECTOR, ".item-content .content")
            content_comment = [elem.text for elem in elems_content]

            elems_skuInfo = driver.find_elements(By.CSS_SELECTOR, ".item-content .skuInfo")
            skuInfo_comment = [elem.text for elem in elems_skuInfo]

            elems_likeCount = driver.find_elements(By.CSS_SELECTOR, ".item-content .bottom .left .left-content")
            like_count = [elem.text for elem in elems_likeCount]

            star_containers = driver.find_elements(By.CSS_SELECTOR, ".container-star.starCtn")
            star_count = [count_rate_star(star_container) for star_container in star_containers]

            df_new = pd.DataFrame(list(zip(name_comment, content_comment, skuInfo_comment, like_count, star_count)),
                                  columns=['name_comment', 'content_comment', 'skuInfo_comment', 'like_count', 'star_count'])
            df_new['Link'] = product_link
            df_new['stt'] = stt  # Thêm cột stt với tên hình ảnh

            global df4
            df4 = pd.concat([df4, df_new], ignore_index=True)
            break  
        except Exception as e:
            driver.execute_script(f"window.scrollBy(0, {scroll_step});")
            sleep(scroll_pause_time)

In [15]:
for i in range(6966, 7100):
    url = df.loc[i, 'links']
    no = df.loc[i, 'no.'] if 'no.' in df.columns else i  
    print(f"Đang xử lý sản phẩm: {url}")
    
    driver.get(url)
    sleep(random.randint(3, 6))  # Chờ trang tải
    
    # Đóng các popup nếu có
    middleware_frame_widget(driver)
    close_sfo_popup(driver)
    baxia_dialog_close(driver)
    content_block (driver)
    
    # Lấy hình ảnh sản phẩm và nhận tên hình ảnh
    image_name = scrape_product_image(driver, url, no, folder_path)
    
    # Nếu không lấy được tên hình ảnh, gán giá trị mặc định
    if image_name is None:
        image_name = f"{no}.jpg"
    
    # Lấy bình luận (tối đa 60 bình luận)
    while True:
        try:
            middleware_frame_widget(driver)
            close_sfo_popup(driver)
            baxia_dialog_close(driver)
            content_block (driver)
            
            scroll_to_comment(driver, ".mod-reviews", url, image_name)

            current_count = len(df4[df4['Link'] == url])
            print(f"Số bình luận hiện tại cho {url}: {current_count}")
            
            if current_count >= 60:
                print(f"Đã lấy đủ 60 bình luận cho {url}.")
                break

            try:
                next_button = driver.find_element(By.CSS_SELECTOR, ".next-pagination-pages button.next-btn.next-btn-normal.next-btn-medium.next-pagination-item.next")
                if next_button.is_enabled():
                    next_button.click()
                    sleep(2)  # Tạm dừng trước khi chuyển trang
                else:
                    print(f"Nút Next Page không khả dụng tại {url}, kết thúc vòng lặp bình luận.")
                    break
            except NoSuchElementException:
                print(f"Không tìm thấy nút Next Page tại {url}, kết thúc vòng lặp bình luận.")
                break
        except Exception as e:
            print(f"Có lỗi xảy ra khi lấy bình luận tại {url}: {e}")
            break

driver.quit()

Đang xử lý sản phẩm: https://www.lazada.vn/products/do-lam-di-chua-bo-la-han-nam-nu-vai-kate-min-nhap-quan-ao-phat-tu-phap-phuc-phat-giao-gian-di-anle-organic-i1999053787.html
Không tìm thấy ảnh cho sản phẩm: https://www.lazada.vn/products/do-lam-di-chua-bo-la-han-nam-nu-vai-kate-min-nhap-quan-ao-phat-tu-phap-phuc-phat-giao-gian-di-anle-organic-i1999053787.html
Vượt quá thời gian 30 giây mà không tìm thấy thêm bình luận, dừng lại.
Số bình luận hiện tại cho https://www.lazada.vn/products/do-lam-di-chua-bo-la-han-nam-nu-vai-kate-min-nhap-quan-ao-phat-tu-phap-phuc-phat-giao-gian-di-anle-organic-i1999053787.html: 0
Không tìm thấy nút Next Page tại https://www.lazada.vn/products/do-lam-di-chua-bo-la-han-nam-nu-vai-kate-min-nhap-quan-ao-phat-tu-phap-phuc-phat-giao-gian-di-anle-organic-i1999053787.html, kết thúc vòng lặp bình luận.
Đang xử lý sản phẩm: https://www.lazada.vn/products/set-bo-thun-dai-theu-chu-bt-thoi-trang-3-size-45-65kg-tuy-chieu-cao-i974818346.html
Đã lưu hình ảnh vào: C:\Use

In [16]:
final_df = df4.groupby('Link').head(60)
final_df.to_csv(r"C:\Users\ASUS\Desktop\T\ĐAN_KLTN\getcomment\combined_data.csv", mode='a', index=False, encoding='utf-8-sig')
print(final_df)

                       name_comment  \
0    Cường L.Chứng nhận đã mua hàng   
1       N***gChứng nhận đã mua hàng   
2       M***nChứng nhận đã mua hàng   
3       M***nChứng nhận đã mua hàng   
4       A***.Chứng nhận đã mua hàng   
..                              ...   
343     D***.Chứng nhận đã mua hàng   
344   Trần M.Chứng nhận đã mua hàng   
345   Oanh T.Chứng nhận đã mua hàng   
346  thanh H.Chứng nhận đã mua hàng   
347    Mai P.Chứng nhận đã mua hàng   

                                       content_comment  \
0    Thiết kế thoải mái và phong cách, Được làm tốt...   
1    shop giao hàng nhanh, đúng và đủ số lượng tron...   
2    Vải mềm và co giãn, Hoàn hảo cho trang phục hà...   
3    Hoàn hảo cho trang phục hàng ngày, Nhiều màu s...   
4    Vải cotton khá mịn, Mặc form vừa y ko cần chỉn...   
..                                                 ...   
343  Vừa vặn và chuẩn size, Giá trị tuyệt vời cho s...   
344  Dễ giặt và bảo quản, Hoàn hảo cho việc nghỉ ng...   
345  Hoàn